In [1]:
import pandas as pd
import os
import sys
import glob
import numpy as np
from xlrd import *
from pathlib import Path
import warnings
warnings.filterwarnings("ignore")
from sklearn.model_selection import train_test_split


In [2]:
#READ CEXCEL FILES AND CONVERT THEM TO CSV FILES AND COMBINE THEM TO ONE FILE
root = '/home/mahnaz/Desktop/iCAIRD_new_slide_labels'
excel_files_dir  = os.path.join(root + '/excel_files')
csv_files_dir = os.path.join(root + '/csv_files')

filenames = os.listdir(excel_files_dir)
for filename in filenames:
    #print('processing {}'.format(filename))
    if '.xls' in filename:
        read_file = pd.read_excel( excel_files_dir + '/'+ filename)
        read_file.to_csv (csv_files_dir +'/' +filename.split('.')[0] + '.csv', index = None, header=True, encoding = 'utf-8-sig')
        
all_filenames = os.listdir(csv_files_dir)
#combine all files in the list
all_files = pd.concat([pd.read_csv(csv_files_dir+ '/'+ f) for f in all_filenames ])
print('There are total {} cervical and endometrial files'.format(len(all_files)))

*** No CODEPAGE record, no encoding_override: will use 'iso-8859-1'
*** No CODEPAGE record, no encoding_override: will use 'iso-8859-1'
*** No CODEPAGE record, no encoding_override: will use 'iso-8859-1'
*** No CODEPAGE record, no encoding_override: will use 'iso-8859-1'
*** No CODEPAGE record, no encoding_override: will use 'iso-8859-1'
*** No CODEPAGE record, no encoding_override: will use 'iso-8859-1'
*** No CODEPAGE record, no encoding_override: will use 'iso-8859-1'
*** No CODEPAGE record, no encoding_override: will use 'iso-8859-1'
*** No CODEPAGE record, no encoding_override: will use 'iso-8859-1'
*** No CODEPAGE record, no encoding_override: will use 'iso-8859-1'
*** No CODEPAGE record, no encoding_override: will use 'iso-8859-1'
*** No CODEPAGE record, no encoding_override: will use 'iso-8859-1'
*** No CODEPAGE record, no encoding_override: will use 'iso-8859-1'
*** No CODEPAGE record, no encoding_override: will use 'iso-8859-1'
*** No CODEPAGE record, no encoding_override: wi

There are total 3849 cervical and endometrial files


In [3]:
cervical = all_files[(all_files.SampleType =='CX') & (all_files.ExcludedFromAnnotation==0)]
print('Total number of cervical train slides after exclusion = {}'.format(len(cervical)))
cervical.head()

Total number of cervical train slides after exclusion = 1930


,AnnotationBatch,ImageName,SampleID,Site,Microtome,Thickness,SampleType,StainingSiteID,StainingProtocol,Cat,SubCat,Revised,ExcludedFromAnnotation,ExclusionReason
0,6,IC-CX-00446-01,IC-CX-00446,RAH,2,4µm,CX,5,neuro (hand staining protocol),High Grade,CIN3,1,0,NaN
1,6,IC-CX-00447-01,IC-CX-00447,QEUH,2,3µm,CX,5,neuro (hand staining protocol),Low Grade,CIN 1,0,0,NaN
2,6,IC-CX-00448-01,IC-CX-00448,QEUH,2,4µm,CX,1,routine H&E,Low Grade,HPV,1,0,NaN
3,6,IC-CX-00451-01,IC-CX-00451,SG,2,4µm,CX,4,paeds protocol,Low Grade,CIN 1,1,0,NaN
4,6,IC-CX-00452-01,IC-CX-00452,RAH,3,3µm,CX,3,paeds protocol,Normal/inflammation,Normal/inflammation,0,0,NaN


In [4]:
cervical = cervical.sort_values(by=['AnnotationBatch','ImageName'])
cervical = cervical.rename(columns={'AnnotationBatch':'batch','ImageName':'Filename','SampleID':'BiobankID', 'Cat':'Category', 'SubCat':'subCategory'})
cervical['Category'] = cervical['Category'].str.lower()
cervical['subCategory'] = cervical['subCategory'].str.lower()

cervical['subCategory'] = cervical['subCategory'].str.replace('/', '_')
cervical['Category'] = cervical['Category'].str.replace('/','_')
cervical['Category'] = cervical['Category'].str.replace(' ','_')
cervical['subCategory'] = cervical['subCategory'].str.replace('cin 1', 'cin1')
cervical['subCategory'] = cervical['subCategory'].str.replace('cin 2', 'cin2')
cervical['subCategory'] = cervical['subCategory'].str.replace('cin 3', 'cin3')
cervical['subCategory'] = cervical['subCategory'].str.replace('cin_1', 'cin1')
cervical['subCategory'] = cervical['subCategory'].str.replace('cin_2', 'cin2')
cervical['subCategory'] = cervical['subCategory'].str.replace('cin_3', 'cin3')
cervical['subCategory'] = cervical['subCategory'].str.replace(' ', '_')
cervical = cervical[['batch','Filename','BiobankID', 'SampleType','Microtome','Thickness', 'Category', 'subCategory', 'StainingSiteID', 'StainingProtocol']]
cervical['Filename'] = cervical['Filename'] +'.isyntax'
cervical = cervical.reset_index(drop=True)
cervical.head()

,batch,Filename,BiobankID,SampleType,Microtome,Thickness,Category,subCategory,StainingSiteID,StainingProtocol
0,1,IC-CX-00001-01.isyntax,IC-CX-00001,CX,3,4µm,normal_inflammation,normal_inflammation,3,paeds protocol
1,1,IC-CX-00002-01.isyntax,IC-CX-00002,CX,2,3µm,low_grade,cin1,2,routine H&E
2,1,IC-CX-00003-01.isyntax,IC-CX-00003,CX,2,4µm,high_grade,cin2,7,muscle (hand staining protocol)
3,1,IC-CX-00004-01.isyntax,IC-CX-00004,CX,1,4µm,normal_inflammation,normal_inflammation,1,routine H&E
4,1,IC-CX-00005-01.isyntax,IC-CX-00005,CX,1,3µm,malignant,cgin,3,paeds protocol


In [5]:
print('Cervical Set Categories Count:\n', cervical['Category'].value_counts())
print('\n')
print('Cervical Set SubCategories Count:\n', cervical['subCategory'].value_counts())

Cervical Set Categories Count:
 low_grade              586
high_grade             490
normal_inflammation    452
malignant              402
Name: Category, dtype: int64


Cervical Set SubCategories Count:
 normal_inflammation    452
hpv                    318
cin1                   268
cin3                   248
cin2                   242
squamous_carcinoma     212
adenocarcinoma          73
cgin                    70
other                   47
Name: subCategory, dtype: int64


In [6]:
cervical_original = pd.read_csv(os.path.join(root + '/iCAIRD_Cervical_Data.csv'))
cervical_original = cervical_original.sort_values(by=['Filename'])
#cervical_original = cervical_original.rename(columns={'subCategory':'SubCategory'})
cervical_original = cervical_original.reset_index(drop=True)
print(len(cervical_original))
cervical_original.head()

2597


,Filename,BiobankID,SampleType,Microtome,Thickness,Category,subCategory,StainingSiteID,StainingProtocol,train/valid/test
0,IC-CX-00001-01.isyntax,IC-CX-00001,CX,3,4µm,normal_inflammation,normal_inflammation,3,paeds protocol,train
1,IC-CX-00002-01.isyntax,IC-CX-00002,CX,2,3µm,malignant,other,2,routine H&E,train
2,IC-CX-00003-01.isyntax,IC-CX-00003,CX,2,4µm,high_grade,cin2,7,muscle (hand staining protocol),train
3,IC-CX-00004-01.isyntax,IC-CX-00004,CX,1,4µm,normal_inflammation,normal_inflammation,1,routine H&E,train
4,IC-CX-00005-01.isyntax,IC-CX-00005,CX,1,3µm,malignant,cgin,3,paeds protocol,train


In [7]:
cervical_original_train = cervical_original[cervical_original['train/valid/test'] == 'train']
cervical_original_train

,Filename,BiobankID,SampleType,Microtome,Thickness,Category,subCategory,StainingSiteID,StainingProtocol,train/valid/test
0,IC-CX-00001-01.isyntax,IC-CX-00001,CX,3,4µm,normal_inflammation,normal_inflammation,3,paeds protocol,train
1,IC-CX-00002-01.isyntax,IC-CX-00002,CX,2,3µm,malignant,other,2,routine H&E,train
2,IC-CX-00003-01.isyntax,IC-CX-00003,CX,2,4µm,high_grade,cin2,7,muscle (hand staining protocol),train
3,IC-CX-00004-01.isyntax,IC-CX-00004,CX,1,4µm,normal_inflammation,normal_inflammation,1,routine H&E,train
4,IC-CX-00005-01.isyntax,IC-CX-00005,CX,1,3µm,malignant,cgin,3,paeds protocol,train
...,...,...,...,...,...,...,...,...,...,...
2589,IC-CX-03132-01.isyntax,IC-CX-03132,CX,2,3µm,high_grade,cin3,3,paeds protocol,train
2590,IC-CX-03134-01.isyntax,IC-CX-03134,CX,2,4µm,high_grade,cin2,2,routine H&E,train
2593,IC-CX-03137-01.isyntax,IC-CX-03137,CX,3,3µm,high_grade,cin3,3,paeds protocol,train
2594,IC-CX-03138-01.isyntax,IC-CX-03138,CX,2,3µm,malignant,adenocarcinoma,2,routine H&E,train


In [8]:
cervical_original_df = cervical_original[['Filename','train/valid/test']]
cervical_original_df

,Filename,train/valid/test
0,IC-CX-00001-01.isyntax,train
1,IC-CX-00002-01.isyntax,train
2,IC-CX-00003-01.isyntax,train
3,IC-CX-00004-01.isyntax,train
4,IC-CX-00005-01.isyntax,train
...,...,...
2592,IC-CX-03136-01.isyntax,test
2593,IC-CX-03137-01.isyntax,train
2594,IC-CX-03138-01.isyntax,train
2595,IC-CX-03139-01.isyntax,train


In [9]:
merged_df = pd.merge(cervical_original_df,cervical, how ='left')
merged_df = merged_df.dropna()
merged_df = merged_df.reset_index(drop=True)
merged_df = merged_df [['batch','Filename','BiobankID', 'SampleType','Microtome','Thickness', 'Category', 'subCategory', 'StainingSiteID', 'StainingProtocol','train/valid/test']]

print('Cervical  Set Main Categories Count:\n', merged_df['Category'].value_counts())
print('\n')
print('Cervical  Set SubCategories Count:\n', merged_df['subCategory'].value_counts())
print('\n')
print('Cervical  Set StainingSiteID Count:\n', merged_df['StainingSiteID'].value_counts())

Cervical  Set Main Categories Count:
 low_grade              586
high_grade             490
normal_inflammation    452
malignant              402
Name: Category, dtype: int64


Cervical  Set SubCategories Count:
 normal_inflammation    452
hpv                    318
cin1                   268
cin3                   248
cin2                   242
squamous_carcinoma     212
adenocarcinoma          73
cgin                    70
other                   47
Name: subCategory, dtype: int64


Cervical  Set StainingSiteID Count:
 4.0    310
2.0    308
1.0    299
3.0    295
7.0    292
5.0    286
8.0     73
6.0     67
Name: StainingSiteID, dtype: int64


In [10]:
cervical_train = merged_df[merged_df['train/valid/test'] == 'train']
cervical_test = merged_df[merged_df['train/valid/test'] == 'test']
print('Cervical Train Set before split to train/valid  Main Categories Count:\n', cervical_train['Category'].value_counts())
print('\n')
print('Cervical Train Set  before split to train/valid  SubCategories Count:\n', cervical_train['subCategory'].value_counts())
print('\n')
print('Cervical Train Set  before split to train/valid  StainingSiteID Count:\n', cervical_train['StainingSiteID'].value_counts())


print('\n')
print('\n')

print('Cervical Test Set Main Categories Count:\n', cervical_test['Category'].value_counts())
print('\n')
print('Cervical Tes Set SubCategories Count:\n', cervical_test['subCategory'].value_counts())
print('\n')
print('Cervical Test Set StainingSiteID Count:\n', cervical_test['StainingSiteID'].value_counts())

Cervical Train Set before split to train/valid  Main Categories Count:
 low_grade              545
high_grade             431
normal_inflammation    399
malignant              363
Name: Category, dtype: int64


Cervical Train Set  before split to train/valid  SubCategories Count:
 normal_inflammation    399
hpv                    293
cin1                   252
cin3                   218
cin2                   213
squamous_carcinoma     190
adenocarcinoma          69
cgin                    60
other                   44
Name: subCategory, dtype: int64


Cervical Train Set  before split to train/valid  StainingSiteID Count:
 2.0    299
4.0    297
1.0    291
3.0    288
7.0    282
5.0    281
Name: StainingSiteID, dtype: int64




Cervical Test Set Main Categories Count:
 high_grade             59
normal_inflammation    53
low_grade              41
malignant              39
Name: Category, dtype: int64


Cervical Tes Set SubCategories Count:
 normal_inflammation    53
cin3                  

In [11]:
from sklearn.model_selection import train_test_split
train, valid = train_test_split(cervical_train, test_size=0.30, random_state=0, stratify=cervical_train[['Category','subCategory','StainingSiteID']])
valid['train/valid/test'] = 'valid'



print('Cervical Train Set Main Categories Count:\n', train['Category'].value_counts())
print('\n')
print('Cervical Train Set SubCategories Count:\n', train['subCategory'].value_counts())
print('\n')
print('Cervical Train Set StainingSiteID Count:\n', train['StainingSiteID'].value_counts())

print('\n')
print('\n')

print('Cervical valid Set Main Categories Count:\n', valid['Category'].value_counts())
print('\n')
print('Cervical valid Set SubCategories Count:\n', valid['subCategory'].value_counts())
print('\n')
print('Cervical valid Set StainingSiteID Count:\n', valid['StainingSiteID'].value_counts())


Cervical Train Set Main Categories Count:
 low_grade              380
high_grade             302
normal_inflammation    279
malignant              255
Name: Category, dtype: int64


Cervical Train Set SubCategories Count:
 normal_inflammation    279
hpv                    204
cin1                   176
cin3                   153
cin2                   149
squamous_carcinoma     134
adenocarcinoma          48
cgin                    41
other                   32
Name: subCategory, dtype: int64


Cervical Train Set StainingSiteID Count:
 2.0    208
4.0    208
1.0    204
3.0    202
7.0    198
5.0    196
Name: StainingSiteID, dtype: int64




Cervical valid Set Main Categories Count:
 low_grade              165
high_grade             129
normal_inflammation    120
malignant              108
Name: Category, dtype: int64


Cervical valid Set SubCategories Count:
 normal_inflammation    120
hpv                     89
cin1                    76
cin3                    65
cin2                  

In [12]:
df = pd.concat([train,valid])
df = df.sort_values(by=['Filename'])
df = df.reset_index(drop=True)
df['batch'] = df['batch'].astype(int)
df['StainingSiteID'] = df['StainingSiteID'].astype(int)

In [13]:
df.to_csv('cervical_Data.csv')
df

,batch,Filename,BiobankID,SampleType,Microtome,Thickness,Category,subCategory,StainingSiteID,StainingProtocol,train/valid/test
0,1,IC-CX-00001-01.isyntax,IC-CX-00001,CX,3.0,4µm,normal_inflammation,normal_inflammation,3,paeds protocol,train
1,1,IC-CX-00002-01.isyntax,IC-CX-00002,CX,2.0,3µm,low_grade,cin1,2,routine H&E,train
2,1,IC-CX-00003-01.isyntax,IC-CX-00003,CX,2.0,4µm,high_grade,cin2,7,muscle (hand staining protocol),train
3,1,IC-CX-00004-01.isyntax,IC-CX-00004,CX,1.0,4µm,normal_inflammation,normal_inflammation,1,routine H&E,valid
4,1,IC-CX-00005-01.isyntax,IC-CX-00005,CX,1.0,3µm,malignant,cgin,3,paeds protocol,train
...,...,...,...,...,...,...,...,...,...,...,...
1733,36,IC-CX-03132-01.isyntax,IC-CX-03132,CX,2.0,3µm,high_grade,cin3,3,paeds protocol,train
1734,36,IC-CX-03134-01.isyntax,IC-CX-03134,CX,2.0,4µm,high_grade,cin2,2,routine H&E,valid
1735,36,IC-CX-03137-01.isyntax,IC-CX-03137,CX,3.0,3µm,high_grade,cin3,3,paeds protocol,train
1736,36,IC-CX-03138-01.isyntax,IC-CX-03138,CX,2.0,3µm,malignant,adenocarcinoma,2,routine H&E,train
